In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45465
2,Huelva,Confirmados PDIA 14 días,270
3,Huelva,Tasa PDIA 14 días,"52,61414346123117"
4,Huelva,Confirmados PDIA 7 días,122
5,Huelva,Tasa PDIA 7 dias,"23,773798156556307"
6,Huelva,Total Confirmados,45668
7,Huelva,Curados,44513
8,Huelva,Fallecidos,416


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45465.0


/tmp/ipykernel_11123/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12557.0


/tmp/ipykernel_11123/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_11123/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_11123/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_11123/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 5753 personas en los últimos 7 días 

Un positivo PCR cada 2569 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45465.0,122.0,270.0,513170.0,23.773798,52.614143,49.0
Huelva-Costa,26915.0,70.0,162.0,289548.0,24.175612,55.949273,26.0
Condado-Campiña,14117.0,44.0,90.0,156231.0,28.163425,57.607005,17.0
Huelva (capital),12557.0,25.0,56.0,143837.0,17.380785,38.932959,12.0
Cartaya,1987.0,18.0,32.0,20083.0,89.628044,159.338744,8.0
Sierra de Huelva-Andévalo Central,4008.0,8.0,18.0,67391.0,11.871021,26.709798,6.0
Moguer,1965.0,12.0,17.0,21867.0,54.877212,77.742717,5.0
Valverde del Camino,841.0,4.0,8.0,12750.0,31.372549,62.745098,4.0
Almonte,2291.0,7.0,18.0,24507.0,28.563268,73.448402,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rosal de la Frontera,76.0,2.0,5.0,1697.0,117.855038,294.637596,2.0
Campofrío,22.0,2.0,2.0,713.0,280.504909,280.504909,0.0
Villarrasa,223.0,3.0,4.0,2211.0,135.685210,180.913614,1.0
Cartaya,1987.0,18.0,32.0,20083.0,89.628044,159.338744,8.0
Paterna del Campo,210.0,3.0,4.0,3457.0,86.780445,115.707261,2.0
Palos de la Frontera,1030.0,3.0,13.0,11742.0,25.549310,110.713677,0.0
Gibraleón,943.0,7.0,13.0,12737.0,54.957996,102.064850,1.0
Bollullos Par del Condado,1286.0,6.0,13.0,14387.0,41.704316,90.359352,2.0
Ayamonte,2077.0,7.0,19.0,21104.0,33.169067,90.030326,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aljaraque,1628.0,2.0,11.0,21474.0,9.313589,51.224737,1.0,0.181818
Punta Umbría,1264.0,1.0,5.0,15355.0,6.512537,32.562683,0.0,0.200000
Palos de la Frontera,1030.0,3.0,13.0,11742.0,25.549310,110.713677,0.0,0.230769
Isla Cristina,3128.0,5.0,18.0,21393.0,23.372131,84.139672,1.0,0.277778
Rociana del Condado,667.0,1.0,3.0,7939.0,12.596045,37.788135,1.0,0.333333
Ayamonte,2077.0,7.0,19.0,21104.0,33.169067,90.030326,1.0,0.368421
Almonte,2291.0,7.0,18.0,24507.0,28.563268,73.448402,3.0,0.388889
Rosal de la Frontera,76.0,2.0,5.0,1697.0,117.855038,294.637596,2.0,0.400000
Huelva-Costa,26915.0,70.0,162.0,289548.0,24.175612,55.949273,26.0,0.432099
